In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import skimage

from src.data_utils import Dataset
from src.network import unet_3d

# tf.enable_eager_execution()

## Setup
- load train and test datasets, create TensorFlow Datasets from them
- define main architecutre params

In [2]:
train_dataset = Dataset.load_dataset(
    '../data/processed/train_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)
test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().batch(1)

In [14]:
# x_train = [p.scans for p in train_dataset.patients.values()]
# y_train = [p.seg for p in train_dataset.patients.values()]
# x_test = [p.scans for p in test_dataset.patients.values()]
# y_test = [p.seg for p in test_dataset.patients.values()]

# i = 28
# inputs = x_train[i]
# outputs = y_train[i]

In [3]:
iterator = tf.data.Iterator.from_structure(
    train_dataset.output_types,
    train_dataset.output_shapes
)
training_init_op = iterator.make_initializer(train_dataset)
test_init_op = iterator.make_initializer(test_dataset)
next_element = iterator.get_next()

In [ ]:
# create the neural network model
logits = unet_3d(next_element[0])

# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# get accuracy
prediction = tf.argmax(logits, axis=-1)
labels = tf.argmax(next_element[1], -1)
equality = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))


init_op = tf.global_variables_initializer()

# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    iou, conf_mat = tf.metrics.mean_iou(labels, tf.cast(prediction, tf.int32), 3)
    sess.run(tf.local_variables_initializer())
    for i in range(epochs):
        l, _, acc, miou = sess.run([loss, optimizer, accuracy, iou])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
            print("Epoch: {}, loss: {:.3f}, training IOU: {:.2f}%".format(i, l, miou * 100))
    # now setup the validation run
    valid_iters = 100
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc = sess.run([accuracy])
        avg_acc += acc[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters))                            

conv1: (?, 32, 128, 128, 16)
conv2: (?, 32, 128, 128, 32)
maxpool layer: (?, 16, 64, 64, 32)
conv1: (?, 16, 64, 64, 32)
conv2: (?, 16, 64, 64, 64)
maxpool layer: (?, 8, 32, 32, 64)
conv1: (?, 8, 32, 32, 64)
conv2: (?, 8, 32, 32, 128)
maxpool layer: (?, 4, 16, 16, 128)
conv1: (?, 4, 16, 16, 128)
conv2: (?, 4, 16, 16, 256)
upconv layer: (?, 8, 32, 32, 256)
concat layer: (?, 8, 32, 32, 128)
up_conv layer1: (?, 8, 32, 32, 128)
up_conv layer2: (?, 8, 32, 32, 128)
upconv layer: (?, 16, 64, 64, 128)
concat layer: (?, 16, 64, 64, 64)
up_conv layer1: (?, 16, 64, 64, 64)
up_conv layer2: (?, 16, 64, 64, 64)
upconv layer: (?, 32, 128, 128, 64)
concat layer: (?, 32, 128, 128, 32)
up_conv layer1: (?, 32, 128, 128, 32)
up_conv layer2: (?, 32, 128, 128, 32)
output layer: (?, 32, 128, 128, 3)
Epoch: 0, loss: 586579.000, training accuracy: 44.97%
Epoch: 0, loss: 586579.000, training IOU: 0.00%
Epoch: 50, loss: 88810.438, training accuracy: 98.76%
Epoch: 50, loss: 88810.438, training IOU: 0.00%
Epoch: 10

In [5]:
train_dataset = Dataset.load_dataset(
    '../data/processed/train_dataset.pckl'
).create_tf_dataset()

# Define network and optimizer
net = unet_3d
opt = tf.train.AdamOptimizer(learning_rate=0.01)
writer = tf.contrib.summary.create_file_writer('tmp')

def loss(net, inputs, labels):
    return tf.reduce_sum(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=net(inputs), labels=labels
        )
    )

def train_step(loss, net, opt, x, y):
    # Perform a single step of optimization
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt.minimize(lambda: loss(net, x, y), global_step=tf.train.get_or_create_global_step())

# Numpy array to keep track of the accuracy
acc_history = np.zeros(50)

# with writer.as_default():
#     with tf.contrib.summary.always_record_summaries():
        
# Loop over the epochs
for epoch in range(50):

    # Initialize the metric
    accuracy = tfe.metrics.Accuracy()

    # For each epoch we shuffle the dataset
    for (xb, yb) in tfe.Iterator(train_dataset.shuffle(40).batch(1)):
        print('============================================')
        # Save the loss on disk
        # tf.contrib.summary.scalar('loss_value', loss(net, xb,yb))

        # Make a training step
        
        train_step(loss, net, opt, xb, yb)
        print('--------------------------------------------')
        # Save the training accuracy on the batch
        accuracy(tf.argmax(net(tf.constant(xb)), axis=1), tf.argmax(tf.constant(yb), axis=1))

    # Save the overall accuracy in our vector
    acc_history[epoch] = accuracy.result().numpy()
            

# At the end, plot the evolution of the training accuracy
import matplotlib.pyplot as plt
plt.figure()
plt.plot(acc_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

conv1: (1, 32, 128, 128, 16)
conv2: (1, 32, 128, 128, 32)
maxpool layer: (1, 16, 64, 64, 32)
conv1: (1, 16, 64, 64, 32)
conv2: (1, 16, 64, 64, 64)
maxpool layer: (1, 8, 32, 32, 64)
conv1: (1, 8, 32, 32, 64)
conv2: (1, 8, 32, 32, 128)
maxpool layer: (1, 4, 16, 16, 128)
conv1: (1, 4, 16, 16, 128)
conv2: (1, 4, 16, 16, 256)
upconv layer: (1, 8, 32, 32, 256)
concat layer: (1, 8, 32, 32, 128)
up_conv layer1: (1, 8, 32, 32, 128)
up_conv layer2: (1, 8, 32, 32, 128)
upconv layer: (1, 16, 64, 64, 128)
concat layer: (1, 16, 64, 64, 64)
up_conv layer1: (1, 16, 64, 64, 64)
up_conv layer2: (1, 16, 64, 64, 64)
upconv layer: (1, 32, 128, 128, 64)
concat layer: (1, 32, 128, 128, 32)
up_conv layer1: (1, 32, 128, 128, 32)
up_conv layer2: (1, 32, 128, 128, 32)
output layer: (1, 32, 128, 128, 3)


TypeError: 'NoneType' object is not subscriptable

In [29]:
outputs.shape

TensorShape([Dimension(24), Dimension(128), Dimension(128), Dimension(3)])

In [92]:
class_weights = tf.constant([[0.1, .45, .45]])
w = tf.gather(class_weights, outputs)
w.shape

TensorShape([Dimension(24), Dimension(128), Dimension(128), Dimension(3), Dimension(3)])

In [58]:
class_weights = tf.constant([[0.1, .45, .45]])
class_weights = tf.broadcast_to(class_weights, outputs.shape.as_list())
w = tf.multiply(tf.cast(outputs, class_weights.dtype), class_weights)

In [ ]:
unweighted_losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels=outputs, logits=y_pred[0,:,:,:,:])
weighted_losses = unweighted_losses * w

In [88]:
unweighted_losses.shape

TensorShape([Dimension(24), Dimension(128), Dimension(128)])

In [76]:

tf.losses.softmax_cross_entropy(outputs, y_pred[0,:,:,:,:], weights=w)

ValueError: weights can not be broadcast to values. values.rank=3. weights.rank=4. values.shape=(24, 128, 128). weights.shape=(24, 128, 128, 3).